In [2]:
#!pip show azureml-train-automl-runtime
#from azureml._common.exceptions import AzureMLAggregatedException

Name: azureml-train-automl-runtime
Version: 1.19.0
Summary: automl
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: azureml-train-automl-client, azureml-dataset-runtime, gensim, scikit-learn, azureml-automl-core, azureml-core, boto3, pandas, onnxruntime, onnx, sklearn-pandas, onnxconverter-common, azureml-interpret, keras2onnx, azureml-automl-runtime, statsmodels, numpy, lightgbm, scipy, azureml-defaults, botocore, azureml-telemetry, smart-open, onnxmltools, skl2onnx, azure-storage-queue, jinja2
Required-by: azureml-train-automl


In [3]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
#ws = Workspace.from_config()
ws = Workspace.get(name="ML_workspace")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ML_workspace
Azure region: eastus2
Subscription id: db3f3e9c-5e3f-47a0-8edc-d3959ba03d04
Resource group: UdacityCourse


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
#ps = RandomParameterSampling(
#    {
#        "solver": choice('liblinear', 'newton-cg','lbfgs')
#    }
#)

ps = RandomParameterSampling(
    {"--C": choice(0.001,0.1,1,2,3),
     '--max_iter' : choice(10,20,30)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",
             compute_target=aml_compute,
             entry_script='train.py',
             )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=15,
                                    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(config = hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [7]:
import joblib
# Get your best run and save the model from that run.
#run.upload_file("outputs/model.joblib", "outputs/model.joblib")
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name='best_run', model_path='outputs/model.joblib')

['--C', '2', '--max_iter', '30']
['azureml-logs/55_azureml-execution-tvmps_4dcc9d18083bd80a553481c639eacbe35b0065d51d580b941abb4efe6505d951_d.txt', 'azureml-logs/65_job_prep-tvmps_4dcc9d18083bd80a553481c639eacbe35b0065d51d580b941abb4efe6505d951_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_4dcc9d18083bd80a553481c639eacbe35b0065d51d580b941abb4efe6505d951_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)

In [9]:
from train import clean_data
from azureml.core.dataset import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = x.join(y)
if "data" not in os.listdir():
    os.mkdir("./data")
df.to_csv("data/clean_data.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

ds = Dataset.Tabular.from_delimited_files(datastore.path('data/clean_data.csv'))

Uploading an estimated of 1 files
Target already exists. Skipping upload for data/clean_data.csv
Uploaded 0 files


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    #experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=2,
    compute_target=aml_compute,
    enable_onnx_compatible_models=True)

In [11]:
# Submit your automl run

automl_run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_17efb64f-02bb-4514-b159-d54f981965de

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because t

In [14]:
best_automl_run = automl_run.get_output()

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.5
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


In [34]:
# Some of the parameter of the automl
best_automl_run[1][1]


PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                               max_

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_model, onnx_model= automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")

Finally we delete the compute target

In [ ]:
aml_compute.delete()
print('Cluster Deleted')
